In [1]:
import tensorflow as tf
from tensorflow.keras import layers , models
import numpy as np

In [2]:
mnist = tf.keras.datasets.mnist
(trainImages , trainLabel) , (testImages , testLabel) = mnist.load_data()
trainImages = trainImages/255.0
testImages  = testImages/255.0

In [3]:
trainImages = trainImages.reshape( -1 , 28 , 28 , 1)
testImages = testImages.reshape(-1, 28 , 28 , 1)

In [4]:
class CustomDenseLayer(tf.keras.layers.Layer):
    def __init__(self, units):
        super(CustomDenseLayer, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.weight = self.add_weight(name='weight',
                                      shape=(input_shape[-1], self.units),
                                      initializer='random_normal',
                                      trainable=True)

    def call(self, inputs):
        output = tf.matmul(inputs, self.weight)
        return output

In [5]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
# model.add(layers.Dense(23 , aactivation = ""))


In [6]:
model.compile(optimizer='adam' , loss  = 'sparse_categorical_crossentropy',  
              metrics = ['accuracy'])

In [7]:
model.fit(trainImages, trainLabel, epochs=1, batch_size=32, validation_data=(testImages, testLabel))


1875/1875 [==============================] - 51s 26ms/step - loss: 0.1361 - accuracy: 0.9587 - val_loss: 0.0554 - val_accuracy: 0.9804


In [8]:
# loss , accuracy  = model.evaluate(testImages , testLabel)
# print(loss)
# print(accuracy)


conv1_weights = model.layers[5].get_weights
# print(conv1_weights)
# var = np.zeros(64)
# for i in range(64):
#     var[i] = i
# from matplotlib import pyplot as plt
# plt.plot(var  , conv1_weights)
# plt.show()
# print(conv1_weights)
print("Shape of the weight matrix in the first Conv2D layer:", conv1_weights)

Shape of the weight matrix in the first Conv2D layer: <bound method Layer.get_weights of <keras.layers.core.dense.Dense object at 0x000002E0763A7460>>


In [9]:
# stdDeviation = np.std(conv1_weights)
# print(stdDeviation)
# MeanVal = np.average(conv1_weights)
# print(MeanVal)

In [10]:
import random
def generateCentrallisedWeight(weightArr):
    ans = np.zeros(len(weightArr))
    stdDeviation = np.std(weightArr)
    MeanVal = np.average(weightArr)
    for i in range(len(weightArr)):
        ans[i] = random.uniform(MeanVal - stdDeviation , MeanVal + stdDeviation)
    return ans


In [11]:
def changeWeights(arr , size):
    ans  = []
    for i in range(size):
        ans.append(generateCentrallisedWeight(arr[i]))
    return ans

In [12]:
optimizedModel = models.Sequential()
optimizedModel.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
optimizedModel.add(layers.MaxPooling2D((2, 2)))
optimizedModel.add(layers.Conv2D(64, (3, 3), activation='relu'))
optimizedModel.add(layers.MaxPooling2D((2, 2)))
optimizedModel.add(layers.Flatten())
optimizedModel.add(layers.Dense(64, activation='relu'))
optimizedModel.add(layers.Dense(10, activation='softmax'))

size =  optimizedModel.layers[5].weights[0].shape[0]
optimizedWeights = changeWeights(optimizedModel.layers[5].weights[0] , size)
optimizedModel.layers[5].weights[0] = optimizedWeights

size =  optimizedModel.layers[6].weights[0].shape[0]
optimizedWeights = changeWeights(optimizedModel.layers[6].weights[0] , size)
optimizedModel.layers[6].weights[0] = optimizedWeights

optimizedModel.compile(optimizer='adam' , loss  = 'sparse_categorical_crossentropy',  metrics = ['accuracy'])
optimizedModel.fit(trainImages, trainLabel, epochs=1, batch_size=32, validation_data=(testImages, testLabel))
y_predict = optimizedModel.predict(testImages)
print(y_predict)
print(testLabel)
# loss, accuracy = model.evaluate(test_images, test_labels)
# print(f'Test accuracy: {accuracy}')


313/313 [==============================] - 3s 9ms/step
[[8.91248177e-08 7.03671716e-08 2.09155805e-05 ... 9.99968171e-01
  2.43797217e-07 7.44526858e-07]
 [1.02734535e-04 4.66749361e-06 9.99855757e-01 ... 3.30238459e-09
  1.66662467e-05 6.68371747e-11]
 [1.14134446e-05 9.99515295e-01 5.65952141e-05 ... 6.36327968e-05
  2.41879461e-05 1.77602953e-06]
 ...
 [1.32797096e-09 1.06182029e-06 1.34065825e-08 ... 3.17949969e-07
  4.14662827e-05 1.16792990e-06]
 [1.72475320e-05 1.13054099e-08 5.13653937e-08 ... 3.26182914e-07
  1.07525522e-02 8.29636065e-07]
 [9.79818105e-06 8.78471624e-08 1.60302206e-05 ... 2.93859470e-09
  4.49196406e-04 2.66679265e-08]]
[7 2 1 ... 4 5 6]


In [13]:
print(testLabel)
# print(testLabel.shape)
# var = np.zeros(64)
# for i in range(64):
#     var[i] = i
# plt.scatter(y_predict , )

[7 2 1 ... 4 5 6]


In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.utils import to_categorical

# Define the custom dense layer
class CustomDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None):
        super(CustomDense, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, input_shape):
        self.kernel = self.add_weight("kernel", shape=(input_shape[-1], self.units))
        self.kernel[0][0] = 1
        print(self.kernel)
        self.bias = self.add_weight("bias", shape=(self.units,))
        
    def call(self, inputs):
        output = tf.matmul(inputs, self.kernel) + self.bias
        if self.activation is not None:
            output = self.activation(output)
        return output

# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((-1, 28, 28, 1)) / 255.0
test_images = test_images.reshape((-1, 28, 28, 1)) / 255.0
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Build the CNN model with the custom dense layer
input_layer = Input(shape=(28, 28, 1))
conv_layer = Conv2D(32, (3, 3), activation='relu')(input_layer)
maxpool_layer = MaxPooling2D((2, 2))(conv_layer)
flatten_layer = Flatten()(maxpool_layer)
custom_dense = CustomDense(units=128, activation='relu')(flatten_layer)
output_layer = Dense(10, activation='softmax')(custom_dense)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile and train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=1, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {accuracy}')


TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [ ]:
train_images